In [21]:
import pandas as pd
import json
import os
import numpy as np
import requests
import sqlalchemy as sql
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from pathlib import Path
import datetime
%matplotlib inline

In [2]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set response URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"
ltc_url = "https://api.alternative.me/v2/ticker/Litecoin/?convert=USD"

In [4]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response,indent=4,sort_keys=True))

{
    "data": {
        "1": {
            "circulating_supply": 18632537,
            "id": 1,
            "last_updated": 1613708578,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 960708488850,
                    "percent_change_1h": 0.181704814070547,
                    "percent_change_24h": -0.834442486523365,
                    "percent_change_7d": 7.65164545579847,
                    "percentage_change_1h": 0.181704814070547,
                    "percentage_change_24h": -0.834442486523365,
                    "percentage_change_7d": 7.65164545579847,
                    "price": 51475.0,
                    "volume_24h": 59159267416
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 18632537,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrenc

In [5]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response,indent=4,sort_keys=True))

{
    "data": {
        "1027": {
            "circulating_supply": 114730302,
            "id": 1027,
            "last_updated": 1613708424,
            "max_supply": 0,
            "name": "Ethereum",
            "quotes": {
                "USD": {
                    "market_cap": 219885536072,
                    "percent_change_1h": 0.0807747499348953,
                    "percent_change_24h": 1.91999576071766,
                    "percent_change_7d": 7.82881911969007,
                    "percentage_change_1h": 0.0807747499348953,
                    "percentage_change_24h": 1.91999576071766,
                    "percentage_change_7d": 7.82881911969007,
                    "price": 1922.06,
                    "volume_24h": 33411668864
                }
            },
            "rank": 2,
            "symbol": "ETH",
            "total_supply": 114730302,
            "website_slug": "ethereum"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurren

In [6]:
# Using the Python requests library, make an API call to access the current price LTC
ltc_response = requests.get(ltc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(ltc_response,indent=4,sort_keys=True))

{
    "data": {
        "2": {
            "circulating_supply": 66517929,
            "id": 2,
            "last_updated": 1613708614,
            "max_supply": 84000000,
            "name": "Litecoin",
            "quotes": {
                "USD": {
                    "market_cap": 15027320094,
                    "percent_change_1h": -0.0343365279964881,
                    "percent_change_24h": -1.604232320061,
                    "percent_change_7d": 22.4191279206678,
                    "percentage_change_1h": -0.0343365279964881,
                    "percentage_change_24h": -1.604232320061,
                    "percentage_change_7d": 22.4191279206678,
                    "price": 226.13,
                    "volume_24h": 9092037454
                }
            },
            "rank": 8,
            "symbol": "LTC",
            "total_supply": 66517929,
            "website_slug": "litecoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrencies

In [7]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response['data']['1']['quotes']['USD']['price']

In [8]:
# Navigate the ETH response object to access the current price of ETH
eth_price = eth_response['data']['1027']['quotes']['USD']['price']

In [9]:
# Navigate the LTC response object to access the current price of LTC
ltc_price = ltc_response['data']['2']['quotes']['USD']['price']

In [23]:
# remember to update date
today_date = str(datetime.datetime.now()).split()[0]
one_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=1*365)).split()[0]
three_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=3*365)).split()[0]
str(datetime.datetime.now()).split(" ")[0]
# Format current date as ISO format
today = pd.Timestamp(today_date, tz="America/New_York").isoformat()
today
# Format start and end dates as ISO format for one year period
start = pd.Timestamp(one_year_ago, tz="America/New_York").isoformat()
end = pd.Timestamp(today_date, tz="America/New_York").isoformat()
# Included for example
# # Get closing prices for FB and TWTR from the last year
# df_portfolio_year = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = start,
#     end = end
# ).df

In [10]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='v2')
display(type(alpaca_api_key)) # test to make sure api key imported successfully, str indicates import successful
display(type(alpaca_secret_key)) # test to make sure api key imported successfully, str indicates import successful

str

str

In [26]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = ['SPY','QQQ','ARKK','AGG','O','VNQ']

# Set timeframe to 1D 
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
start_date = pd.Timestamp(today,tz='America/New_York').isoformat() # remember to update date
end_date = pd.Timestamp(today,tz='America/New_York').isoformat() # remember to update date

In [25]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_barset(tickers,timeframe,start=start_date,end=end_date).df

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

AGG                                      ARKK  \
                             open    high     low  close   volume      open   
2021-02-18 00:00:00-05:00  116.14  116.28  116.03  116.2  3910324  149.0487   

                                                             ...     SPY  \
                             high     low   close    volume  ...    open   
2021-02-18 00:00:00-05:00  150.22  146.09  149.15  10568885  ...  389.59   

                                                                VNQ         \
                              high     low   close    volume   open   high   
2021-02-18 00:00:00-05:00  391.515  387.74  390.75  52370380  88.47  88.79   

                                                   
                              low  close   volume  
2021-02-18 00:00:00-05:00  88.255  88.41  3116553  

[1 rows x 30 columns]

In [13]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = prices_df['AGG']['close'].item()

In [14]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
spy_close_price = prices_df['SPY']['close'].item()

In [15]:
# Access the closing price for QQQ from the Alpaca DataFrame
# Converting the value to a floating point number
qqq_close_price = prices_df['QQQ']['close'].item()

In [16]:
# Access the closing price for ARKK from the Alpaca DataFrame
# Converting the value to a floating point number
arkk_close_price = prices_df['ARKK']['close'].item()

In [17]:
# Access the closing price for O from the Alpaca DataFrame
# Converting the value to a floating point number
o_close_price = prices_df['O']['close'].item()

In [18]:
# Access the closing price for VNQ from the Alpaca DataFrame
# Converting the value to a floating point number
vnq_close_price = prices_df['VNQ']['close'].item()
print(vnq_close_price)

88.41


In [20]:
# Get user info
user_info = pd.read_csv('generic_user.csv')

In [28]:
# Start to build portfolio options
user_info

,user_age,user_number
0,23,1


In [31]:
age = user_info['user_age']

In [36]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user doesn't want crypto
portfolio_allocation_stocks_no_crypto = 100-age
portfolio_allocation_stocks_no_crypto

0    77
Name: user_age, dtype: int64

In [37]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user does want crypto


In [41]:
# Create basic portfolio models if user wants less customization
# Order is based [bonds, stocks, crypto]
conservative_portfolio = [.7,.3,0]
conservative_portfolio_crypto = [.7,.2,.1]
moderate_portfolio = [.5,.5,0]
moderate_portfolio_crypto = [.4,.4,.2]
aggressive_portfolio = [.3,.7,0]
aggressive_portfolio_crypto = [.2,.6,.2]
ultra_aggressive = [0,.7,.3]

In [ ]:
# GOAL: finish MVP by Tuesday, full by Thursday
# Build portfolios with each allocation
# Run monte carlos and graphs
# Finish CLI
# Test for bugs
# 